In [1]:
import numpy as np
import glob
import os
import pandas as pd
import scipy.signal as signal
import mne
import torch
from torch.utils.data import DataLoader, Dataset
from mne import preprocessing, Epochs
import utils
import matplotlib.pyplot as plt

## Data pre-processing

In [2]:
#create dataframe from CSV files
eeg_file_path = 'data/eeg_data_A/'
eog_file_path = 'data/eeg_data_B/'
#eeg_dataframe_A.head()
#eeg_dataframe_A = utils.format_df(eeg_dataframe_A)
#eeg_dataframe_A.head()
#eeg_dataframe = utils.gdf_to_df(file_path + 'A01T.gdf')


### Band-pass filtering (noise reduction)

In [3]:
eeg_files = glob.glob(os.path.join(eeg_file_path, '*.gdf'))
eog_files = glob.glob(os.path.join(eog_file_path + '*.gdf'))
eeg_channels = 25
eog_channels = 6
method = 'fastica'
raw_eeg_objects = []  # list to store raw objects
raw_eog_objects = []

eog_epoch_objects = []
eeg_epoch_objects = []
rename_dict = {
    'EOG:ch01': 'EOG001',
    'EOG:ch02': 'EOG002',
    'EOG:ch03': 'EOG003'
}


for file in eeg_files + eog_files:
    raw = mne.io.read_raw_gdf(file, preload=True)
    if file in eog_files:
        print(file)
        # Rename EOG channels and set channel types
        raw.rename_channels(rename_dict)
        raw.set_channel_types({'EOG001': 'eog', 'EOG002': 'eog', 'EOG003': 'eog'})

        # Create EOG epochs
        eog_epochs = mne.preprocessing.create_eog_epochs(raw, baseline=(-0.5, -0.2))
        eog_epoch_objects.append(eog_epochs)
        raw_eog_objects.append(raw)
    else:
        print(file)
        events, event_id = mne.events_from_annotations(raw)
        tmin, tmax = -0.2, 0.5
        eeg_epochs = mne.Epochs(raw, events, event_repeated='merge', event_id=None, tmin=tmin, tmax=tmax, baseline=(None, 0), preload=True)
        eeg_epoch_objects.append(eeg_epochs)
        raw_eeg_objects.append(raw)
    raw.filter(l_freq=1.0, h_freq=40.0, verbose=False)



Extracting EDF parameters from /run/media/thomas/hdd/repos/deep-learning/midterm_project/data/eeg_data_A/A09E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 675097  =      0.000 ...  2700.388 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


data/eeg_data_A/A09E.gdf
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
585 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 585 events and 176 original time points ...
1 bad epochs dropped


NameError: name 'eog_epochs' is not defined

Convert to PyTorch Tensors

In [ ]:
eeg_arrays = []
eog_arrays = []

for raw in raw_eeg_objects:
    data = raw.get_data()
    eeg_arrays.append(data)

for raw in raw_eog_objects:
    data = raw.get_data()
    eog_arrays.append(data)

eeg_arrays = utils.pad_arrays(eeg_arrays)
eog_arrays = utils.pad_arrays(eog_arrays)

tensor_eeg = torch.tensor(eeg_arrays,dtype=torch.float32)
tensor_eog = torch.tensor(eog_arrays,dtype=torch.float32)

print(tensor_eeg.shape)
print(tensor_eog.shape)

torch.Size([18, 25, 687792])
torch.Size([40, 6, 787729])
